# **YoloV5(PyTorch) 모델 실습**

## **1. 사전 환경 세팅**

### **1-1. PyTorch 정상 설치 확인**

In [ ]:
%cd /workspace

In [ ]:
import torch
print(torch.__version__)

### **1-2. Pre-trained weight로 추론 진행**

In [ ]:
# pretrained weight 인 yolov5l.pt 를 활용해 data/bus.jpg 이미지에 대해 추론 진행
!python detect.py --weights yolov5l.pt --source /workspace/data/images/bus.jpg
                                            #    img.jpg                         # image
                                            #    vid.mp4                         # video
                                            #    screen                          # screenshot
                                            #    path/                           # directory
                                            #    list.txt                        # list of images
                                            #    list.streams                    # list of streams
                                            #    'path/*.jpg'                    # glob
                                            #    'https://youtu.be/LNwODJXcvt4'  # YouTube
                                            #    'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream

In [ ]:
# 추론 결과 시각화를 위한 모듈 로드
from IPython.display import Image, display
import glob
import os

# runs/detects/ 폴더 내 가장 최신 exp 확인
exp_dirs = sorted(glob.glob('runs/detect/exp*'), key=os.path.getmtime)
latest_exp_dir = exp_dirs[-1] if exp_dirs else None

# 폴더 내 추론된 이미지가 있을 경우 출력
if latest_exp_dir:
    result_images = glob.glob(os.path.join(latest_exp_dir, '*.jpg'))
    
    if result_images:
        display(Image(filename=result_images[0]))
    else:
        print("No result images found in the latest directory.")
else:
    print("No 'exp*' directories found.")

## **2. PASCAL VOC 데이터 활용 모델 학습**

### **2-1. 데이터 로드 및 전처리**

In [ ]:
import os
import sys
from pathlib import Path
import torch
from utils.general import check_dataset, check_img_size, increment_path, colorstr
from utils.torch_utils import select_device
from utils.dataloaders import create_dataloader

# 프로젝트 루트 디렉토리를 PYTHONPATH에 추가 (노트북에서 실행 시 필요)
FILE = Path(__file__).resolve() if "__file__" in globals() else Path.cwd()
ROOT = FILE.parents[0]
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

In [ ]:
# 데이터셋 로드 및 전처리 함수 정의
def load_data(data_path, img_size, batch_size, hyp):
    """
    데이터셋을 로드하고 전처리를 수행하는 함수
    Args:
        data_path (str): 데이터셋 yaml 파일 경로
        img_size (int): 이미지 크기
        batch_size (int): 배치 크기
        hyp (dict): 하이퍼파라미터 딕셔너리
    Returns:
        train_loader (DataLoader): 학습 데이터 로더
        dataset (Dataset): 데이터셋 객체
    """
    # 데이터셋 경로 및 정보 로드
    data_dict = check_dataset(data_path)  
    train_path, val_path = data_dict['train'], data_dict['val']  # 학습 및 검증 데이터 경로

    # 데이터 로더 생성
    train_loader, dataset = create_dataloader(
        train_path,
        img_size,
        batch_size,
        stride=32,  # YOLO 모델 기본 stride
        hyp=hyp,
        augment=True,  # 데이터 증강
        cache=None,    # 데이터 캐싱 비활성화
        rect=False,    # 직사각형 크기 맞추기 비활성화
        rank=-1,       # DDP 비활성화
        workers=8,     # 데이터 로딩 워커 수
        image_weights=False,  # 이미지 가중치 비활성화
        prefix=colorstr('train: ')
    )
    
    return train_loader, dataset

### **2-2. 모델 및 하이퍼파라미터 정의**

In [ ]:
# yolo 모델 호출
from models.yolo import Model

def define_model(cfg, nc, hyp, device):
    """
    YOLOv5 모델을 정의하는 함수
    Args:
        cfg (str): 모델 구성 파일 경로 (.yaml)
        nc (int): 클래스 수
        hyp (dict): 하이퍼파라미터 딕셔너리
        device (torch.device): 사용할 디바이스 ('cpu' 또는 'cuda')
    Returns:
        model (Model): YOLOv5 모델 객체
    """
    # 모델 생성 및 하이퍼파라미터 설정
    model = Model(cfg, ch=3, nc=nc).to(device)  # YOLO 모델 초기화
    model.hyp = hyp  # 하이퍼파라미터 설정
    return model

In [ ]:
# 하이퍼파라미터 정의
def get_hyperparameters():
    """
    YOLOv5 모델 학습에 필요한 하이퍼파라미터를 정의하는 함수
    Returns:
        hyp (dict): 하이퍼파라미터 딕셔너리
    """
    hyp = {
        'lr0': 0.01,  # 초기 학습률
        'momentum': 0.937,  # 모멘텀
        'weight_decay': 0.0005,  # 가중치 감쇠
        'box': 0.05,  # 박스 손실 가중치
        'cls': 0.5,  # 클래스 손실 가중치
        'obj': 1.0,  # 객체 손실 가중치
        'iou_t': 0.2,  # IoU 임계값
        'anchor_t': 4.0,  # 앵커 임계값
        'fl_gamma': 0.0,  # Focal Loss 감마 값
        'hsv_h': 0.015,  # HSV 색상 변화
        'hsv_s': 0.7,  # HSV 채도 변화
        'hsv_v': 0.4,  # HSV 명도 변화
        'degrees': 0.0,  # 회전 각도
        'translate': 0.1,  # 이동 변환
        'scale': 0.5,  # 스케일 변환
        'shear': 0.0,  # 왜곡 변환
        'cls_pw': 1.0,  # 클래스 손실 양성 가중치
        'obj_pw': 1.0,  # 객체 손실 양성 가중치
        'mosaic': 1.0,  # 모자이크 데이터 증강 사용 여부
        'copy_paste': 0.0,  # Copy-paste 증강 사용 여부
        'perspective': 0.0,  # 원근 변환
        'mixup': 0.0,  # Mixup 증강 사용 여부
        'flipud': 0.0,  # 수직 플립 확률
        'fliplr': 0.5,  # 수평 플립 확률
    }
    return hyp

### **2-3. 모델 학습**

In [186]:
# 모델 학습 함수 정의 및 학습 진행

def train(hyp, opt, device):
    # 경로 설정
    save_dir = Path(opt.save_dir)  # 결과 저장 경로 설정
    epochs, batch_size, weights, data = opt.epochs, opt.batch_size, opt.weights, opt.data

    # 데이터셋 및 모델 configuration 호출
    data_dict = check_dataset(data)  # data.yaml 호출(VOC.yaml)
    train_path, val_path = data_dict['train'], data_dict['val']
    nc = int(data_dict['nc'])  # 클래스 개수
    names = data_dict['names']  # 클래스 명

    # 모델
    model = Model(opt.cfg, ch=3, nc=nc).to(device)  # 모델 신규 정의
    model.hyp = hyp  # 사전 정의 하이퍼파라미터 반영
    imgsz = check_img_size(opt.imgsz, s=32)  # 이미지 크기 검증

    # 데이터
    train_loader, dataset = create_dataloader(train_path,
                                              imgsz,
                                              batch_size,
                                              32,
                                              hyp=hyp,
                                              augment=True,
                                              cache=None,
                                              rect=False,
                                              rank=-1,
                                              workers=8,
                                              image_weights=False,
                                              prefix=colorstr('train: ')
                                              )

    # optimizer, 손실함수 정의
    optimizer = torch.optim.SGD(model.parameters(), lr=hyp['lr0'], momentum=hyp['momentum'], weight_decay=hyp['weight_decay'])
    compute_loss = ComputeLoss(model)

    # 학습 루프
    for epoch in range(epochs):
        model.train()
        for imgs, targets, paths, _ in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{epochs}'): # 데이터 루트 폴더 내 이미지 list에 대한 루프
            imgs = imgs.to(device).float() / 255.0  # 이미지 정규화
            targets = targets.to(device)

            # Forward pass
            pred = model(imgs)
            loss, loss_items = compute_loss(pred, targets)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f'Epoch {epoch + 1}/{epochs} finished.')

    # 가중치를 저장할 때 모델의 전체 상태를 저장하는 예시
    ckpt = {
        'epoch': epoch,
        'best_fitness': None,  # 최상의 피트니스 점수 (필요시 추가)
        'model': model,  # 전체 모델 객체를 저장
        'optimizer': optimizer.state_dict(),
        'hyp': hyp,  # 하이퍼파라미터
        'names': names,  # 클래스 이름
        'ema': None,  # EMA 객체 (필요시 추가)
        'updates': None,  # EMA 업데이트 수
    }

    torch.save(ckpt, save_dir / 'last.pt')

# 학습 옵션 설정 클래스
class Options:
    def __init__(self):
        self.weights = ''  # 초기 가중치 경로, 기본적으로는 빈 문자열 (초기화되지 않음)
        self.cfg = 'models/yolov5s.yaml'  # 모델 구성 파일 경로
        self.data = '/workspace/data/VOC.yaml'  # 데이터셋 구성 파일 경로
        self.epochs = 1  # 총 학습 에폭 수
        self.batch_size = 16  # 배치 크기
        self.imgsz = 640  # 학습 및 검증 시 사용할 이미지 크기 (픽셀)
        self.save_dir = 'runs/train'  # 학습 결과 저장 디렉토리

# 학습 시작
if __name__ == '__main__':
    opt = Options()
    device = select_device('0' if torch.cuda.is_available() else 'cpu')
    opt.save_dir = increment_path(Path(opt.save_dir) / 'exp')
    opt.save_dir.mkdir(parents=True, exist_ok=True)

    hyp = get_hyperparameters()
    train(hyp, opt, device)


YOLOv5 🚀 2024-8-31 Python-3.10.9 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12042MiB)

Overriding model.yaml nc=80 with nc=20

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  3    625152  models.common.C3                        [256, 256, 3]                 
  7         

Epoch 1/1 finished.


## **3. 신규 학습 weight를 활용한 추론**

In [189]:
# 신규 학습 weight 인 yolov5s.pt 를 활용해 data/bus.jpg 이미지에 대해 추론 진행
!python detect.py --weights /workspace/runs/train/exp17/last.pt --source /workspace/data/images/bus.jpg --data data/VOC.yaml
                                            #    img.jpg                         # image
                                            #    vid.mp4                         # video
                                            #    screen                          # screenshot
                                            #    path/                           # directory
                                            #    list.txt                        # list of images
                                            #    list.streams                    # list of streams
                                            #    'path/*.jpg'                    # glob
                                            #    'https://youtu.be/LNwODJXcvt4'  # YouTube
                                            #    'rtsp://example.com/media.mp4'  # RTSP, RTMP, HTTP stream

detect2: weights=['/workspace/runs/train/exp17/last.pt'], source=/workspace/data/images/bus.jpg, data=data/VOC.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-361-gc5ffbbf1 Python-3.10.9 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12042MiB)

Traceback (most recent call last):
  File "/workspace/detect2.py", line 437, in <module>
    main(opt)
  File "/workspace/detect2.py", line 432, in main
    run(**vars(opt))
  File "/opt/conda/lib/python3.10/site-packages/torch/utils/_contextlib.py", line 115, in decorate_context
    return func(*args, **kwargs)
  File "/workspace/detect2.py", line 166, in run
    model = Detec

In [ ]:
# 추론 결과 시각화를 위한 모듈 로드
from IPython.display import Image, display
import glob
import os

# runs/detects/ 폴더 내 가장 최신 exp 확인
exp_dirs = sorted(glob.glob('runs/detect/exp*'), key=os.path.getmtime)
latest_exp_dir = exp_dirs[-1] if exp_dirs else None

# 폴더 내 추론된 이미지가 있을 경우 출력
if latest_exp_dir:
    result_images = glob.glob(os.path.join(latest_exp_dir, '*.jpg'))
    
    if result_images:
        display(Image(filename=result_images[0]))
    else:
        print("No result images found in the latest directory.")
else:
    print("No 'exp*' directories found.")